<a href="https://colab.research.google.com/github/takumi19910112/google_colab/blob/main/gemini_for_eval.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# ドライブをマウントする
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!ls /content/drive/MyDrive/

'1_1 アジェンダ新規作成.drawio'
 1997年課題ロードマップ.gslides
 2023-05-10.mov
'202405月分勤怠率 のコピー.gsheet'
 2024_11_30.ipynb
'2024910_all (5).csv'
'３Q 目標.gdoc'
 ４月１２日（水）.gdoc
'４月１２日（水） のコピー.gdoc'
'8 27　大戸さん.gjam'
'9 11.ipynb'
 aaa.km
 api.gdoc
'APP_NAME=DMM_CRANE_CMS.gdoc'
 Atcoder-daily-ac-checker.gscript
 Atcoder-daily-ac-checker.gsheet
 bus.jpg
 CGI
'Chrome から保存'
'Colab Notebooks'
 competency.txt
 cursorで開く未来.gslides
 diagram.png
 diagram.svg
 Distribution20241221_all
 docker-dmm-crane
 elyza-tasks-100.csv
 elyza-tasks-100-TV_0.jsonl
 elyza_tasks_input_only.jsonl
 elyza_tasks_modified.jsonl
 elyza_tasks_with_output_0.jsonl
 elyza_tasks_with_output_example.jsonl
 embeddings.npy
 face-demographics-walking.mp4
' FE241:特定の手順を行うと、新登場ステーションがお気に入りに表示されない'
'FE253:[低]iPadのsafariで、「ベル」か「チケット」ボタンで遷移後にブラウザバックすると画面が拡大表示されている'
 fine-tuned_and_rag.ipynb
 fine-tuning_0.ipynb
 fine-tuning_1201.ipynb
 fr.gdoc
 gemini_for_eval.ipynb
 G検定まとめ.xlsx
 G検定　数理統計解説　アンケート.gform
 huggingface
 IMG_0548.HEIC
 IMG_05

In [ ]:
!pwd

/content


In [ ]:
import json

# データセットの読み込み
datasets = []
with open("/content/drive/MyDrive/ELYZA-tasks-100-TV-results-20241202.jsonl", "r") as f:
    item = ""
    for line in f:
        line = line.strip()
        item += line
        if item.endswith("}"):
            datasets.append(json.loads(item))
            item = ""

# 新しいデータ構造に変換
structured_datasets = {}
for dataset in datasets:
    task_id = dataset['task_id']
    input_text = dataset['input']
    output_text = dataset['output']

    structured_datasets[task_id] = {
        'input': input_text,
        'output': output_text,
    }

# 結果の表示（確認用）
# for task_id, content in structured_datasets.items():
#     print(f"Task ID: {task_id}")
#     print(f"Input: {content['input']}")
#     print(f"Output: {content['output']}")
#     print("\n---\n")


In [ ]:
# geminiの導入
import os
from google.colab import userdata
import google.generativeai as genai
# Google APIキーを取得
google_api_key = userdata.get('GOOGLE_API_KEY')
os.environ["GOOGLE_API_KEY"] = google_api_key  # APIキーを環境変数に設定
genai.configure(api_key=os.environ["GOOGLE_API_KEY"])
model = genai.GenerativeModel("gemini-1.5-flash")
response = model.generate_content("ハロー。元気ですか？")
print(response.text)

元気です！ あなたはどうですか？



In [ ]:
# プロンプトテンプレートの読み込み
prompt= """
問題, 正解例, 採点基準, 言語モデルが生成した回答が与えられます。

# 指示
「採点基準」と「正解例」を参考にして、回答を1,2,3,4,5の5段階で採点し、数字のみを出力してください。

# 問題
{input_text}

# 正解例
{output_text}

# 採点基準
基本的な採点基準
- 1点: 誤っている、 指示に従えていない
- 2点: 誤っているが、方向性は合っている
- 3点: 部分的に誤っている、 部分的に合っている
- 4点: 合っている
- 5点: 役に立つ

基本的な減点項目
- 不自然な日本語: -1点
- 部分的に事実と異なる内容を述べている: -1点
- 「倫理的に答えられません」のように過度に安全性を気にしてしまっている: 2点にする
"""

# LLMを利用する際にプロンプトを生成して採点
def evaluate(entry):
    input_data = entry["input"]
    expected_output = entry["output"]
    # プロンプトにデータを埋め込む
    evaluation_prompt = prompt.format(input_text=input_data, output_text=expected_output)
    # LLMに問い合わせを行い、評価を得る
    response = model.generate_content(evaluation_prompt)

    # 数字のみの評価を取得して返す
    score = response.text.strip()
    return score

total_score = 0
count = 0

# 各データセット項目を評価
for entry in datasets:
    score = evaluate(entry)
    # スコアを数値に変換し、合計に追加（エラー処理を含めることを推奨）
    try:
        score = int(score)
        total_score += score
        count += 1
    except ValueError:
        # print(f"Task ID: {entry['task_id']} - Score '{score}' is not a valid number")
        continue

# 平均スコアを計算
if count > 0:
    average_score = total_score / count
    print(f"Average Score: {average_score:.2f}")
else:
    print("No valid scores to calculate an average.")

Average Score: 3.48
